In [ ]:
# Read dataset
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler

df = pd.read_csv('data/selected_data.csv')
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
df = df.iloc[:,:-1]

cols = df.columns.values
df = pd.DataFrame(StandardScaler().fit_transform(df), columns=cols)